https://www.kaggle.com/prashant111/catboost-classifier-in-python

In [ ]:
!nvidia-smi

Sat Dec  4 02:58:32 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install catboost
!pip install scikit-learn
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension

     |████████████████████████████████| 76.3 MB 1.3 MB/s 
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta
import re

def season(month):
    if month == 12 or month == 1 or month == 2:
        return "winter"
    if 3 <= month <= 5:
        return "spring"
    if 6 <= month <= 8:
        return "summer"
    if 9 <= month <= 11:
        return "autumn"


def day_part(hour):
    if 12 <= hour < 18:
        return "day"
    if 18 <= hour <= 23:
        return "evening"
    if 6 <= hour < 12:
        return "morning"
    if 0 <= hour <= 6:
        return "night"


def real_time(shift, date):
    diff_time = str(shift[3:]) if len(str(shift)) > 3 else 0
    if type(diff_time) == type(0):
        return date
    elif diff_time[0:1] == "+":
        return date + timedelta(hours=int(diff_time[1:]))
    elif diff_time[0:1] == "-":
        return date - timedelta(hours=int(diff_time[1:]))
    return date

def city_type(population):
    if population < 10000:
        return 'village'
    elif population < 100000:
        return 'small'
    elif population < 500000:
        return 'medium'
    elif population < 1000000:
        return 'big'
    elif population >= 1000000:
        return 'million'
    return 'small'


In [4]:
mer = pd.read_csv('/content/drive/MyDrive/mer_train_fin3.csv', sep=';', parse_dates=['real_time'])

In [6]:
mer.head()

,Segment,bundle,oblast,city,os,osv,real_time,weekday,season,day_part,os_global,category,subcategory
0,5,com.aim.rallyhum,Свердловская область,Екатеринбург,android,8.1.0,2021-09-19 11:33:15,6,autumn,morning,8.0,Games,Racing
1,5,com.higgs.tilemaster3d,Тюмень,Тюмень,android,10.0,2021-08-04 16:45:58,2,summer,day,10.0,Games,Puzzle
2,5,solitaire.patience.card.games.klondike.free,Смоленская область,Смоленск,android,10.0,2021-08-05 05:13:17,3,summer,night,10.0,Games,Card
3,5,com.gameinsight.gplay.mmanor,Санкт-Петербург,Санкт-Петербург,android,5.1,2021-09-17 00:59:08,4,autumn,night,5.0,Games,Adventure
4,5,com.amanotes.beathopper,Московская область,Дмитров,android,6.0.1,2021-08-04 15:17:20,2,summer,day,6.0,Games,Music


In [5]:
mer = mer.drop("Unnamed: 0", axis=1)

In [13]:
mer.shape[0]

10047930

In [12]:
mer = mer[mer['subcategory'].notna()]

In [ ]:
mer = mer.drop_duplicates()

In [14]:
mer['category'] = mer['category'].astype(str)
mer['subcategory'] = mer['subcategory'].astype(str)
mer['bundle'] = mer['bundle'].astype(str)
mer['season'] = mer['season'].astype(str)
mer['oblast'] = mer['oblast'].astype(str)
mer['city'] = mer['city'].astype(str)
mer['os'] = mer['os'].astype(str)
mer['osv'] = mer['osv'].astype(str)
mer['day_part'] = mer['day_part'].astype(str)
# mer['city_type'] = mer['city_type'].astype(str)
mer['os_global'] = mer['os_global'].astype(str)

In [15]:
X = mer.drop("Segment", axis=1)
y = mer["Segment"]

In [16]:
cat_features = [0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11]
print(cat_features)

[0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11]


In [ ]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(
    iterations=1500, 
    learning_rate=0.5, 
    task_type="GPU",
    devices='0',
    loss_function='MultiClass'
    # custom_loss=['AUC', 'Accuracy']
    #loss_function='CrossEntropy'
)

In [17]:
from catboost import CatBoostClassifier
model = CatBoostClassifier()

model.load_model("model3")

In [18]:
model.fit(
    X, y,
    cat_features=cat_features,
    # eval_set=(X_val, y_val),
)

0:	learn: 1.1648030	total: 555ms	remaining: 13m 51s
1:	learn: 1.0278412	total: 1.14s	remaining: 14m 16s
2:	learn: 0.9728985	total: 1.69s	remaining: 14m 4s
3:	learn: 0.9202727	total: 2.2s	remaining: 13m 43s
4:	learn: 0.8932949	total: 2.73s	remaining: 13m 36s
5:	learn: 0.8722802	total: 3.29s	remaining: 13m 40s
6:	learn: 0.8584575	total: 3.81s	remaining: 13m 31s
7:	learn: 0.8515016	total: 4.28s	remaining: 13m 18s
8:	learn: 0.8450407	total: 4.78s	remaining: 13m 11s
9:	learn: 0.8395283	total: 5.23s	remaining: 12m 59s
10:	learn: 0.8312030	total: 5.79s	remaining: 13m 4s
11:	learn: 0.8279224	total: 6.26s	remaining: 12m 56s
12:	learn: 0.8232054	total: 6.71s	remaining: 12m 47s
13:	learn: 0.8209346	total: 7.12s	remaining: 12m 35s
14:	learn: 0.8194469	total: 7.53s	remaining: 12m 25s
15:	learn: 0.8149372	total: 8.02s	remaining: 12m 24s
16:	learn: 0.8111714	total: 8.51s	remaining: 12m 22s
17:	learn: 0.8082138	total: 9.04s	remaining: 12m 24s
18:	learn: 0.8070548	total: 9.51s	remaining: 12m 21s
19:	le

In [ ]:
importances = model.feature_importances_
feature_list = list(X.columns)
feature_results = pd.DataFrame({'feature': feature_list,'importance': importances})
feature_results = feature_results.sort_values('importance',ascending = False).reset_index(drop=True)
feature_results.head(15)

,feature,importance
0,real_time,46.695880
1,day_part,11.411876
2,oblast,9.177510
3,weekday,7.216374
4,bundle,7.210209
5,osv,6.811352
6,season,4.058769
7,os,2.248721
8,subcategory,1.940255
9,city,1.751989


In [19]:
model.save_model("model5")

# from_file = CatBoostClassifier()

# from_file.load_model("model")

In [2]:
mer2 = pd.read_csv('/content/drive/MyDrive/mer_test_fin.csv', parse_dates=['real_time'])

In [3]:
mer2['category'] = mer2['category'].astype(str)
mer2['subcategory'] = mer2['subcategory'].astype(str)
mer2['bundle'] = mer2['bundle'].astype(str)
mer2['season'] = mer2['season'].astype(str)
mer2['oblast'] = mer2['oblast'].astype(str)
mer2['city'] = mer2['city'].astype(str)
mer2['os'] = mer2['os'].astype(str)
mer2['osv'] = mer2['osv'].astype(str)
mer2['day_part'] = mer2['day_part'].astype(str)
# mer['city_type'] = mer['city_type'].astype(str)
mer2['os_global'] = mer2['os_global'].astype(str)

In [6]:
mer2.head()

,bundle,oblast,city,os,osv,real_time,weekday,season,day_part,os_global,category,subcategory
0,com.allgoritm.youla,Свердловская область,Екатеринбург,android,10.0,2021-09-19 19:31:33,6,autumn,evening,10.0,Shopping,Shopping
1,com.ChocochocoStd.RapBattle,Ямало-Ненецкий АО,Новый Уренгой,android,10.0,2021-08-03 19:13:17,1,summer,evening,10.0,nan,nan
2,1387897651,Москва,Москва,ios,14.4.0,2021-09-17 15:54:00,4,autumn,day,14.0,Games,Board
3,com.cooking.family.diary.fever.food.city.craze...,Краснодарский край,Краснодар,android,9.0.0,2021-07-05 23:34:59,0,summer,evening,9.0,Games,Simulation
4,com.pedometer.stepcounter.tracker,Татарстан,Набережные Челны,android,10.0,2021-07-08 15:15:21,3,summer,day,10.0,Health & Fitness,Health & Fitness


In [5]:
mer2 = mer2.drop("Unnamed: 0", axis=1)

In [7]:
from catboost import CatBoostClassifier, Pool
model = CatBoostClassifier()

model.load_model("model5")

In [8]:
pred_test = model.predict(data=mer2)

In [9]:
list_classes = []
for i in pred_test:
    list_classes.append(i[0])

In [10]:
mer2['Segment'] = list_classes

In [11]:
mer2.head()

,bundle,oblast,city,os,osv,real_time,weekday,season,day_part,os_global,category,subcategory,Segment
0,com.allgoritm.youla,Свердловская область,Екатеринбург,android,10.0,2021-09-19 19:31:33,6,autumn,evening,10.0,Shopping,Shopping,3
1,com.ChocochocoStd.RapBattle,Ямало-Ненецкий АО,Новый Уренгой,android,10.0,2021-08-03 19:13:17,1,summer,evening,10.0,nan,nan,3
2,1387897651,Москва,Москва,ios,14.4.0,2021-09-17 15:54:00,4,autumn,day,14.0,Games,Board,5
3,com.cooking.family.diary.fever.food.city.craze...,Краснодарский край,Краснодар,android,9.0.0,2021-07-05 23:34:59,0,summer,evening,9.0,Games,Simulation,3
4,com.pedometer.stepcounter.tracker,Татарстан,Набережные Челны,android,10.0,2021-07-08 15:15:21,3,summer,day,10.0,Health & Fitness,Health & Fitness,4


In [12]:
mer2.to_csv('/content/drive/MyDrive/test_pred.csv', encoding='utf-8')

In [ ]:
y[0]

In [ ]:
count_true = 0
pred = model.predict(data=X)
count = 0
for i in range(n):
    try:
        count += 1
        if pred[i][0] == y[i]:
          count_true += 1
    except:
        continue
print(count_true)
print(count_true / count)

1264381
0.12585914791956998
